# 5.8 学習済みモデルの保存 p.331

## TensorFlow スタイルでモデルの保存と復元をプログラミングする

<ol>
    <li> 学習に使用したモデルと同じモデルを生成する。</li>
    <li> 生成したモデルに compile() メソッドでオプティマイザー、損失関数を設定してコンパイルを行う。</li>
    <li> 保存した重みを読み込む。</li>
</ol>

<code>Model</code>クラスには重みを読み込むための <code>load_weights()</code> が用意されているので、これを使ってもよい。

## 4章 4.3 で作成した Fashion-MNIST 学習結果を保存する。

<a href="../html/tf2_book4_ch04_03.html">4.3 章</a> で行ったモデルの学習を再び行う。        

In [1]:
# データの読み込み
from tensorflow import keras
(x_train, t_train), (x_test, t_test) = keras.datasets.fashion_mnist.load_data() 

In [2]:
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [3]:
# 4-03-6 (28, 28) の画像データを(784)のベクトルに変換してから、正規化する

x_train_rs = x_train.reshape((-1, 28*28)).astype('float32') / 255
x_test_rs = x_test.reshape((-1, 28*28)).astype('float32') / 255

print(x_train_rs.shape)
print(x_test_rs.shape)

(60000, 784)
(10000, 784)


In [9]:
# 正解ラベルを one-hot vector 化する
t_train_cat = keras.utils.to_categorical(t_train, 10)
t_test_cat = keras.utils.to_categorical(t_test, 10)

In [4]:
# 4-03-9 モデルの定義
from tensorflow import keras

class MLP(keras.Model):

    def __init__(self, hidden_dim, output_dim):
        super().__init__()
        self.l1 = keras.layers.Dense(hidden_dim, activation='relu')
        self.l2 = keras.layers.Dense(output_dim, activation='softmax')
        
    def call(self, x):
        h = self.l1(x)
        y = self.l2(h)
        return y

In [5]:
# 4-03-10 損失関数の定義
# p.220

cce = keras.losses.CategoricalCrossentropy()

def loss(t, y):
    return cce(t, y)

In [6]:
# 4-03-11 Gradient Descentによるparameter のupdate処理
import tensorflow as tf

optimizer = keras.optimizers.SGD(learning_rate=0.1)

train_loss = keras.metrics.Mean()
train_acc = keras.metrics.CategoricalAccuracy()

def train_step(x, t):
    with tf.GradientTape() as tape:
        outputs = model(x)
        tmp_loss = loss(t, outputs)
    grads = tape.gradient(tmp_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_loss(tmp_loss) # 損失をMeanオブジェクトに記録する
    train_acc(t, outputs) # 精度を記録する
    return tmp_loss

In [12]:
# 4-03-12 訓練データと検証データの用意
from sklearn.model_selection import train_test_split

x_tr, x_val, t_tr, t_val = train_test_split(x_train_rs, t_train_cat, test_size=0.2)

In [11]:
%%time
# 4-03-13 学習の実行
# p.223

from sklearn.utils import shuffle

epochs = 100
batch_size = 64
steps = x_train.shape[0] // batch_size

model = MLP(256, 10)

for epoch in range(epochs):
    x_, t_ = shuffle(x_tr, t_tr, random_state=1)
    
    for step in range(steps):
        start = step * batch_size
        end = start + batch_size
        tmp_loss = train_step(x_[start:end], t_[start:end])
        
    if (epoch + 1) % 10 == 0:
        print(f'epoch: {epoch+1} train_loss: {train_loss.result():.4f} train_acc: {train_acc.result():.4f}')
        
model.summary()

epoch: 10 train_loss: 0.2922 train_acc: 0.8691
epoch: 20 train_loss: 0.2426 train_acc: 0.8910
epoch: 30 train_loss: 0.2117 train_acc: 0.9053
epoch: 40 train_loss: 0.1883 train_acc: 0.9164
epoch: 50 train_loss: 0.1692 train_acc: 0.9256
epoch: 60 train_loss: 0.1531 train_acc: 0.9333
epoch: 70 train_loss: 0.1395 train_acc: 0.9398
epoch: 80 train_loss: 0.1278 train_acc: 0.9452
epoch: 90 train_loss: 0.1178 train_acc: 0.9499
epoch: 100 train_loss: 0.1092 train_acc: 0.9538
Model: "mlp_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              multiple                  200960    
_________________________________________________________________
dense_3 (Dense)              multiple                  2570      
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 0
_________________________________________________________________
Wall time: 9min 46s


In [13]:
# 4-03-14 検証データによるモデルの評価
#p.225

val_acc = tf.keras.metrics.CategoricalAccuracy()

val_preds = model(x_val)
val_acc.update_state(t_val, val_preds)

validation_loss = loss(t_val, val_preds)

print(f'validation_loss: {validation_loss:.4f} validation_acc: {val_acc.result():.4f}')

validation_loss: 0.1642 validation_acc: 0.9501


In [14]:
# 4-03-15 テストデータによるモデルの評価
test_acc = tf.keras.metrics.CategoricalAccuracy()

test_preds = model(x_test_rs)
test_acc.update_state(t_test_cat, test_preds)

test_loss = loss(t_test_cat, test_preds)

print(f'test_loss: {test_loss:.4f} test_acc: {test_acc.result():.4f}')

test_loss: 0.5270 test_acc: 0.8829


## Fashion-MNIST の学習結果を保存し復元する p.332

In [15]:
# 5-08-1 Fashion-MNIST の学習後の重みを保存する
# p.332

model.save_weights('model_weights_ch04_03', save_format='tf')

In [16]:
# 5-08-2 モデルを再生成してコンパイル、保存した重みを読み込む
# p.332

# 学習に使用したモデルと同じ構造のモデルを生成する
loaded_model = MLP(256, 10)

loaded_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

loaded_model.load_weights('model_weights_ch04_03')

In [17]:
# 5-08-2 再現したモデルを検証データで確認する
val_acc = tf.keras.metrics.CategoricalAccuracy()

val_preds = loaded_model(x_val)
val_acc.update_state(t_val, val_preds)
val_loss = loss(t_val, val_preds)

print(f'val_loss: {val_loss:.4f}  val_acc: {val_acc.result():.4f}')

val_loss: 0.1642  val_acc: 0.9501


In [19]:
# 5-08-3 再現したモデルをテストデータで確認する
test_acc = tf.keras.metrics.CategoricalAccuracy()

test_preds = loaded_model(x_test_rs)
test_acc.update_state(t_test_cat, test_preds)
test_loss = loss(t_test_cat, test_preds)

print(f'val_loss: {test_loss:.4f}  test_acc: {test_acc.result():.4f}')

val_loss: 0.5270  test_acc: 0.8829
